**FCN - Inspect Weights of a Trained Model

This notebook includes code and visualizations to test, debug, and evaluate the Mask R-CNN model.

## Build FCN Model and display summary

In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:95% !important; }</style>"))
%matplotlib inline
%load_ext autoreload
%autoreload 2
import sys, os, random, pprint
sys.path.append('../..')
import tensorflow as tf
import keras.backend as KB
import numpy as np
import skimage.io
import matplotlib.pyplot as plt
import mrcnn.visualize as visualize
import mrcnn.utils as utils
from mrcnn.datagen         import data_generator, load_image_gt, data_gen_simulate
from mrcnn.utils           import mask_string, parse_image_meta, apply_box_deltas_tf
from mrcnn.prep_notebook   import build_newshapes_config, build_coco_config
from mrcnn.coco            import prep_coco_dataset, CocoConfig ## CocoDataset, CocoConfig, CocoInferenceConfig, evaluate_coco, build_coco_results
import mrcnn.model_fcn     as fcn_modellib
from mrcnn.utils           import log
pp = pprint.PrettyPrinter(indent=2, width=100)
np.set_printoptions(linewidth=100,precision=4,threshold=1000, suppress = True)

## Notebook Preferences
from mrcnn.utils import convertHtmlToPdf

# Device to load the neural network on Useful if you're training a model on the same 
# machine, in which case use CPU and leave the GPU for training.
DEVICE = "/cpu:1"  # /cpu:0 or /gpu:0

Using TensorFlow backend.


In [2]:
# WEIGHT_FILE = 'F:/models/train_fcn8_coco_adam/fcn20181109T0000/fcn_2298.h5'
WEIGHT_FILE = 'last'
##------------------------------------------------------------------------------------
## Parse command line arguments
##------------------------------------------------------------------------------------
parser = utils.command_line_parser()
input_parms  =" --batch_size     1  "
input_parms +=" --mrcnn_logs_dir train_mrcnn_coco_subset "
# input_parms +=" --fcn_logs_dir   train_fcn8_bce " 
# input_parms +=" --fcn_logs_dir   train_fcn8_coco_adam "
input_parms +=" --fcn_logs_dir   train_fcn8L2_BCE_subset "
input_parms +=" --mrcnn_model    last "
# input_parms +=" --fcn_model      " + WEIGHT_FILE 
input_parms +=" --fcn_model      last "
input_parms +=" --fcn_layer      all"
input_parms +=" --fcn_arch       fcn8l2 " 
input_parms +=" --sysout         screen "
input_parms +=" --scale_factor   4"
input_parms +=" --coco_classes       78 79 80 81 82 44 46 47 48 49 50 51 34 35 36 37 38 39 40 41 42 43 10 11 13 14 15 "

args = parser.parse_args(input_parms.split())
utils.display_input_parms(args)


   Arguments passed :
   --------------------
   batch_size                     1
   coco_classes                   [78, 79, 80, 81, 82, 44, 46, 47, 48, 49, 50, 51, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 10, 11, 13, 14, 15]
   epochs                         1
   evaluate_method                1
   fcn_arch                       FCN8L2
   fcn_layers                     ['all']
   fcn_logs_dir                   train_fcn8L2_BCE_subset
   fcn_losses                     fcn_BCE_loss
   fcn_model                      last
   last_epoch                     0
   lr                             0.001
   mrcnn_exclude_layers           None
   mrcnn_layers                   ['mrcnn', 'fpn', 'rpn']
   mrcnn_logs_dir                 train_mrcnn_coco_subset
   mrcnn_model                    last
   new_log_folder                 False
   opt                            ADAGRAD
   scale_factor                   4
   steps_in_epoch                 1
   sysout                         SCREEN
   val_ste

In [3]:
from mrcnn.prep_notebook import build_fcn_inference_pipeline
mrcnn_model, fcn_model = build_fcn_inference_pipeline(args = args,verbose = 1)


--> Execution started at: 01-30-2019 @ 11:45:25
    Tensorflow Version: 1.8.0   Keras Version : 2.1.6 
>>> Initialize Paths
 Linx  Linux

   Arguments passed :
   --------------------
   batch_size                     1
   coco_classes                   [78, 79, 80, 81, 82, 44, 46, 47, 48, 49, 50, 51, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 10, 11, 13, 14, 15]
   epochs                         1
   evaluate_method                1
   fcn_arch                       FCN8L2
   fcn_layers                     ['all']
   fcn_logs_dir                   train_fcn8L2_BCE_subset
   fcn_losses                     fcn_BCE_loss
   fcn_model                      last
   last_epoch                     0
   lr                             0.001
   mrcnn_exclude_layers           None
   mrcnn_layers                   ['mrcnn', 'fpn', 'rpn']
   mrcnn_logs_dir                 train_mrcnn_coco_subset
   mrcnn_model                    last
   new_log_folder                 False
   opt                     


>>> Detection Layer (Inference Mode)
    Detection Layer : call()  <class 'list'> 4
    rpn_proposals_roi              :  shape: (1, ?, ?)             KB.shape:(None, 1000, 4)       Keras Tensor: True
    mrcnn_class.shape              :  shape: (?, 1000, 81)         KB.shape:(None, 1000, 81)      Keras Tensor: True
    mrcnn_bboxes.shape             :  shape: (?, 1000, 81, 4)      KB.shape:(None, 1000, 81, 4)   Keras Tensor: True
    input_image_meta               :  shape: (?, ?)                KB.shape:(None, None)          Keras Tensor: True
    Detections                     :  shape: <unknown>             KB.shape:(None, 200, 7)        Keras Tensor: True

--------------------------------
>>>  CHM Inference Layer  
--------------------------------
  > CHM Inference Layer: call  <class 'list'> 1
     detections.shape     : (None, 200, 7)

  > build_predictions Inference mode ()
    config image shape     :  [1024 1024    3] h: 1024 w: 1024
    Detection Max Instacnes:  200
    num

   FCN Block 33 shape is               :  (None, 64, 64, 256)
   FCN Block 34 (Max pooling) shape is :  (?, 32, 32, 256)
   FCN Block 41 shape is               :  (None, 32, 32, 512)
   FCN Block 42 shape is               :  (None, 32, 32, 512)
   FCN Block 43 shape is               :  (None, 32, 32, 512)
   FCN Block 44 (Max pooling) shape is :  (?, 16, 16, 512)
   FCN Block 51 shape is               :  (None, 16, 16, 512)
   FCN Block 52 shape is               :  (None, 16, 16, 512)
   FCN Block 53 shape is               :  (None, 16, 16, 512)
   FCN Block 54 (Max pooling) shape is :  (None, 8, 8, 512)

   --- FCN32 ----------------------------
   FCN fully connected 1 (fc1) shape   :  (None, 8, 8, 4096)
   FCN fully connected 2 (fc2) shape   :  (None, 8, 8, 4096)
   FCN conv2d (fcn32_deconv2D) shape   :  (?, 8, 8, 81)  keras_tensor  True

   --- FCN16 ----------------------------
   FCN scorePool4 (Conv2D(Pool4)) shape is                   :  (None, 16, 16, 81)    keras_tensor  True

    alt_scores_2_norm(by_image)    :  shape: (?, 3)                KB.shape:(None, 3)             Keras Tensor: False
    fcn_scores_dense               :  shape: (?, 24)               KB.shape:(None, 24)            Keras Tensor: False
    seq_ids                        :  shape: (?,)                  KB.shape:(None,)               Keras Tensor: False
    sscatter_ids                   :  shape: (?, 2)                KB.shape:(None, 2)             Keras Tensor: False
    fcn_scores_by_class            :  shape: (1, 81, 200, 24)      KB.shape:(1, 81, 200, 24)      Keras Tensor: False
    complete                       
    * fcn_heatmap shape:           :  shape: (?, 256, 256, 81)     KB.shape:(None, 256, 256, 81)  Keras Tensor: True
    * fcn_scores shape :           :  shape: (1, 81, 200, 24)      KB.shape:(1, 81, 200, 24)      Keras Tensor: True
 self.keras_model.losses :  21
0      Tensor("block3_conv2/add:0", shape=(), dtype=float32)
1      Tensor("block1_conv2/add:0", shape=(), dt

## Set weight files

In [13]:
### MSE ERRORS
# DIR_WEIGHTS =  '/home/kbardool/models_coco/train_fcn8L2_MSE_subset/fcn20190112T0000' 
# files       = ['fcn_0001.h5', 'fcn_0100.h5', 'fcn_0220.h5', 'fcn_0464.h5',  'fcn_0542.h5',
#                'fcn_0690.h5', 'fcn_1015.h5', 'fcn_1228.h5', 'fcn_1568.h5',  'fcn_1603.h5', 'fcn_1806.h5']
# files       = [ 'fcn_0001.h5', 'fcn_0100.h5', 'fcn_0220.h5', 'fcn_0464.h5',
#                 'fcn_0690.h5', 'fcn_1015.h5', 'fcn_1228.h5', 'fcn_1568.h5', 
#                 'fcn_1603.h5', 'fcn_1806.h5',
#                 'fcn_1065.h5', 'fcn_1095.h5', 'fcn_1108.h5',
#                 'fcn_1612.h5', 'fcn_1673.h5', 'fcn_2330.h5', 'fcn_3348.h5',
#                 'fcn_3742.h5', 'fcn_3816.h5', 'fcn_4345.h5']


## BCE Weight files 
# DIR_WEIGHTS =  '/home/kbardool/models_coco/train_fcn8L2_BCE_subset/fcn20181221T0000' 
# files   = ['fcn_0001.h5', 'fcn_0026.h5', 'fcn_0162.h5', 'fcn_0350.h5',
#            'fcn_0584.h5', 'fcn_0657.h5', 'fcn_0950.h5']

# DIR_WEIGHTS =  '/home/kbardool/models_coco/train_fcn8L2_BCE_subset/fcn20181230T0000' 
# files   = ['fcn_1065.h5', 'fcn_1095.h5', 'fcn_1108.h5']

DIR_WEIGHTS =  '/home/kbardool/models_coco/train_fcn8L2_BCE_subset/fcn20190120T0000' 
files   = ['fcn_1612.h5', 'fcn_1673.h5', 'fcn_2330.h5', 'fcn_3348.h5',
           'fcn_3742.h5', 'fcn_3816.h5', 'fcn_4345.h5']


In [16]:
# for FILE_IDX  in [0,1,2,3,4,5,6]:

#     weights_path = os.path.join(DIR_WEIGHTS  , files[FILE_IDX])
#     if not os.path.isfile(weights_path):
#         continue
#     print("Loading weights ", weights_path)
#     fcn_model.load_model_weights(weights_path)
#     # Pick layer types to display
#     a = visualize.display_weight_histograms(fcn_model,width=15,height=4, filename = files[FILE_IDX])
#     weights_histogram = os.path.join(DIR_WEIGHTS  , 'histogram_'+files[FILE_IDX]+'.png')
#     a.savefig(weights_histogram)

## Load Weights - 1

In [17]:
FILE_IDX = -1

weights_path = os.path.join(DIR_WEIGHTS  , files[FILE_IDX])
print("Loading weights ", weights_path)
fcn_model.load_model_weights(weights_path)

Loading weights  /home/kbardool/models_coco/train_fcn8L2_BCE_subset/fcn20190120T0000/fcn_4345.h5
-----------------------------------------------
 Load Model with init parm: [ /home/kbardool/models_coco/train_fcn8L2_BCE_subset/fcn20190120T0000/fcn_4345.h5 ]
-----------------------------------------------
 ---> Explicit weight file
>>> load_weights() from : /home/kbardool/models_coco/train_fcn8L2_BCE_subset/fcn20190120T0000/fcn_4345.h5
    Weights file loaded: /home/kbardool/models_coco/train_fcn8L2_BCE_subset/fcn20190120T0000/fcn_4345.h5 
FCN  MODEL Load weight file COMPLETE 


### Review Weight Stats - 1st weight file

In [18]:
# Show stats of all trainable weights    
a = visualize.display_weight_stats(fcn_model)
weights_stats = os.path.join(DIR_WEIGHTS  , 'stats_'+files[FILE_IDX]+'.pdf')
# utils.convertHtmlToPdf(a, weights_stats) 

LAYER,WEIGHT NAME,LAYER TYPE,SHAPE,MIN,MAX,STD
0,block1_conv1/kernel:0,Conv2D,"(3, 3, 81, 64)",-0.3050740957,+0.1416187882,+0.0182258748
0,block1_conv1/bias:0,Conv2D,"(64,)",-0.0513798445,+0.0295557808,+0.0122315269
1,block1_conv2/kernel:0,Conv2D,"(3, 3, 64, 64)",-0.1975537986,+0.1305981427,+0.0297307093
1,block1_conv2/bias:0,Conv2D,"(64,)",-0.0425476953,+0.0239294823,+0.0129021415
2,block2_conv1/kernel:0,Conv2D,"(3, 3, 64, 128)",-0.1559223086,+0.1407191604,+0.0177510940
2,block2_conv1/bias:0,Conv2D,"(128,)",-0.0453735329,+0.0209344625,+0.0123706711
3,block2_conv2/kernel:0,Conv2D,"(3, 3, 128, 128)",-0.1054843590,+0.0884783491,+0.0107584568
3,block2_conv2/bias:0,Conv2D,"(128,)",-0.0384271257,+0.0229410194,+0.0091476850
4,block3_conv1/kernel:0,Conv2D,"(3, 3, 128, 256)",-0.1285320669,+0.0974962786,+0.0073803854
4,block3_conv1/bias:0,Conv2D,"(256,)",-0.0400719717,+0.0302388072,+0.0105000166


### Histograms of Weights - 1st weight file

In [ ]:
# Pick layer types to display
a = visualize.display_weight_histograms(fcn_model,width=15,height=4, filename = files[FILE_IDX])
weights_histogram = os.path.join(DIR_WEIGHTS  , 'histogram_'+files[FILE_IDX]+'.png')
a.savefig(weights_histogram)

## Load Weights - 2nd weight file

In [ ]:
FILE_IDX = 11

weights_path = os.path.join(DIR_WEIGHTS  , files[FILE_IDX])
print("Loading weights ", weights_path)
fcn_model.load_model_weights(weights_path)

### Review Weight Stats - 2nd weights file

In [ ]:
# Show stats of all trainable weights    
a = visualize.display_weight_stats(fcn_model)
weights_stats = os.path.join(DIR_WEIGHTS  , 'stats_'+files[FILE_IDX]+'.pdf')
# utils.convertHtmlToPdf(a, weights_stats) 

### Histograms of Weights - 2nd weights file

In [ ]:
# Pick layer types to display
a = visualize.display_weight_histograms(fcn_model,width=15,height=4, filename = files[FILE_IDX])
weights_histogram = os.path.join(DIR_WEIGHTS  , 'histogram_'+files[FILE_IDX]+'.png')
a.savefig(weights_histogram)

## Load Weights - 3rd weight file

### load

In [ ]:
FILE_IDX = 18

weights_path = os.path.join(DIR_WEIGHTS  , files[FILE_IDX])
print("Loading weights ", weights_path)
fcn_model.load_model_weights(weights_path)

### Review Weight Stats - 3rd weight file

In [ ]:
# Show stats of all trainable weights    
a = visualize.display_weight_stats(fcn_model)
weights_stats = os.path.join(DIR_WEIGHTS  , 'stats_'+files[FILE_IDX]+'.pdf')
# utils.convertHtmlToPdf(a, weights_stats) 
# Pick layer types to display
a = visualize.display_weight_histograms(fcn_model,width=15,height=4, filename = files[FILE_IDX])
weights_histogram = os.path.join(DIR_WEIGHTS  , 'histogram_'+files[FILE_IDX]+'.png')
a.savefig(weights_histogram)

### Histograms of Weights - 3rd weight file

In [ ]:
# a = visualize.display_weight_histograms(fcn_model, filename = files[2])
# weights_histogram = os.path.join(DIR_WEIGHTS  , 'histogram_'+files[2]+'.png')
# a.savefig(weights_histogram)

## Load Weights - 4rd weight file

In [ ]:
weights_path = os.path.join(DIR_WEIGHTS  , files[3])
print("Loading weights ", weights_path)
fcn_model.load_model_weights(weights_path) 

### Review Weight Stats - 4th weight file

In [ ]:
# Show stats of all trainable weights    
# _ = visualize.display_weight_stats(fcn_model)

### Histograms of Weights

In [ ]:
# a = visualize.display_weight_histograms(fcn_model, filename = files[3])
# weights_histogram = os.path.join(DIR_WEIGHTS  , 'histogram_'+files[3]+'.png')
# a.savefig(weights_histogram)

## Load Weights - 5th weight file

In [ ]:
print(len(files))
weights_path = os.path.join(DIR_WEIGHTS  , files[4])
fcn_model.load_model_weights(weights_path) 

### Review Weight Stats - 5th weight file

In [ ]:
# Show stats of all trainable weights    
# visualize.display_weight_stats(fcn_model)

### Histograms of Weights - 5th weight file

In [ ]:
# a = visualize.display_weight_histograms(fcn_model, filename = files[4])
# weights_histogram = os.path.join(DIR_WEIGHTS  , 'histogram_'+files[4]+'.png')
# a.savefig(weights_histogram)

## Load Weights - 6th weight file

In [ ]:
weights_path = os.path.join(DIR_WEIGHTS  , files[5])
fcn_model.load_model_weights(weights_path) 

### Review Weight Stats - 6th weight file

In [ ]:
# Show stats of all trainable weights    
# _ = visualize.display_weight_stats(fcn_model)

### Histograms of Weights - 6th weight file

In [ ]:
# a = visualize.display_weight_histograms(fcn_model, filename = files[5])
# weights_histogram = os.path.join(DIR_WEIGHTS  , 'histogram_'+files[5]+'.png')
# a.savefig(weights_histogram)

## Load Weights - 7th weight file

In [ ]:
weights_path = os.path.join(DIR_WEIGHTS  , files[6])
fcn_model.load_model_weights(weights_path) 

### Review Weight Stats - 7th weight file

In [ ]:
# Show stats of all trainable weights    
# _ = visualize.display_weight_stats(fcn_model)

### Histograms of Weights - 7th weight file

In [ ]:
# a = visualize.display_weight_histograms(fcn_model, filename = files[6])
# weights_histogram = os.path.join(DIR_WEIGHTS  , 'histogram_'+files[6]+'.png')
# a.savefig(weights_histogram)

## Load Weights - 8th weight file

In [ ]:
weights_path = os.path.join(DIR_WEIGHTS  , files[7])
fcn_model.load_model_weights(weights_path) 

### Review Weight Stats - 8th weight file

In [ ]:
# Show stats of all trainable weights    
# _ = visualize.display_weight_stats(fcn_model)

### Histograms of Weights - 8th weight file

In [ ]:
# a = visualize.display_weight_histograms(fcn_model, filename = files[7])
# weights_histogram = os.path.join(DIR_WEIGHTS  , 'histogram_'+files[7]+'.png')
# a.savefig(weights_histogram)

## Load Weights - 9th weight file

In [ ]:
weights_path = os.path.join(DIR_WEIGHTS  , files[8])
fcn_model.load_model_weights(weights_path) 

### Review Weight Stats - 9th weight file

In [ ]:
# Show stats of all trainable weights    
# _ = visualize.display_weight_stats(fcn_model)

### Histograms of Weights - 9th weight file

In [ ]:
# a = visualize.display_weight_histograms(fcn_model, filename = files[8])
# weights_histogram = os.path.join(DIR_WEIGHTS  , 'histogram_'+files[8]+'.png')
# a.savefig(weights_histogram)